# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 15 2023 8:30AM,257414,16,SHL-9161575,"SHL Pharma, LLC",Released
1,Feb 15 2023 8:30AM,257397,10,0086387970,ISDIN Corporation,Released
2,Feb 15 2023 8:30AM,257397,10,0086387977,ISDIN Corporation,Released
3,Feb 15 2023 8:30AM,257397,10,0086387971,ISDIN Corporation,Released
4,Feb 15 2023 8:30AM,257397,10,0086387972,ISDIN Corporation,Released
5,Feb 15 2023 8:30AM,257397,10,0086386514,ISDIN Corporation,Released
6,Feb 15 2023 8:30AM,257397,10,0086386516,ISDIN Corporation,Released
7,Feb 15 2023 8:30AM,257397,10,0086386515,ISDIN Corporation,Released
8,Feb 15 2023 8:30AM,257397,10,0086387498,ISDIN Corporation,Released
9,Feb 15 2023 8:30AM,257397,10,0086387496,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,257409,Released,1
52,257410,Released,2
53,257412,Released,2
54,257413,Released,1
55,257414,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257409,NaN,NaN,1.0
257410,NaN,NaN,2.0
257412,NaN,NaN,2.0
257413,NaN,NaN,1.0
257414,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257314,0.0,0.0,3.0
257320,0.0,0.0,1.0
257321,0.0,0.0,1.0
257331,0.0,0.0,31.0
257336,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257314,0,0,3
257320,0,0,1
257321,0,0,1
257331,0,0,31
257336,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257314,0,0,3
1,257320,0,0,1
2,257321,0,0,1
3,257331,0,0,31
4,257336,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257314,,,3
1,257320,,,1
2,257321,,,1
3,257331,,,31
4,257336,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC"
1,Feb 15 2023 8:30AM,257397,10,ISDIN Corporation
11,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation
17,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation
71,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation
80,Feb 15 2023 8:30AM,257404,10,"Nextsource Biotechnology, LLC"
81,Feb 15 2023 8:30AM,257405,10,Eywa Pharma Inc.
82,Feb 15 2023 8:30AM,257395,10,Emerginnova
84,Feb 15 2023 8:30AM,257401,10,"Snap Medical Industries, LLC"
85,Feb 15 2023 8:18AM,257413,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC",,,1
1,Feb 15 2023 8:30AM,257397,10,ISDIN Corporation,,,14
2,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,,,5
3,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,,,55
4,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,,,5
5,Feb 15 2023 8:30AM,257404,10,"Nextsource Biotechnology, LLC",,,1
6,Feb 15 2023 8:30AM,257405,10,Eywa Pharma Inc.,,,1
7,Feb 15 2023 8:30AM,257395,10,Emerginnova,,1,1
8,Feb 15 2023 8:30AM,257401,10,"Snap Medical Industries, LLC",,,1
9,Feb 15 2023 8:18AM,257413,19,"AdvaGen Pharma, Ltd",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC",1,,
1,Feb 15 2023 8:30AM,257397,10,ISDIN Corporation,14,,
2,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,5,,
3,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,55,,
4,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5,,
5,Feb 15 2023 8:30AM,257404,10,"Nextsource Biotechnology, LLC",1,,
6,Feb 15 2023 8:30AM,257405,10,Eywa Pharma Inc.,1,,
7,Feb 15 2023 8:30AM,257395,10,Emerginnova,1,1,
8,Feb 15 2023 8:30AM,257401,10,"Snap Medical Industries, LLC",1,,
9,Feb 15 2023 8:18AM,257413,19,"AdvaGen Pharma, Ltd",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC",1,,
1,Feb 15 2023 8:30AM,257397,10,ISDIN Corporation,14,,
2,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,5,,
3,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,55,,
4,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC",1.0,NaN,NaN
1,Feb 15 2023 8:30AM,257397,10,ISDIN Corporation,14.0,NaN,NaN
2,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,5.0,NaN,NaN
3,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,55.0,NaN,NaN
4,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC",1.0,0.0,0.0
1,Feb 15 2023 8:30AM,257397,10,ISDIN Corporation,14.0,0.0,0.0
2,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,5.0,0.0,0.0
3,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,55.0,0.0,0.0
4,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,9265125,306.0,12.0,28.0
12,257406,1.0,0.0,0.0
15,257354,6.0,0.0,1.0
16,257414,1.0,0.0,0.0
19,2316519,13.0,1.0,0.0
20,257336,1.0,0.0,0.0
22,772178,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,9265125,306.0,12.0,28.0
1,12,257406,1.0,0.0,0.0
2,15,257354,6.0,0.0,1.0
3,16,257414,1.0,0.0,0.0
4,19,2316519,13.0,1.0,0.0
5,20,257336,1.0,0.0,0.0
6,22,772178,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,306.0,12.0,28.0
1,12,1.0,0.0,0.0
2,15,6.0,0.0,1.0
3,16,1.0,0.0,0.0
4,19,13.0,1.0,0.0
5,20,1.0,0.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,306.0
1,12,Released,1.0
2,15,Released,6.0
3,16,Released,1.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,28.0,0.0,1.0,0.0,0.0,0.0,0.0
Executing,12.0,0.0,0.0,0.0,1.0,0.0,0.0
Released,306.0,1.0,6.0,1.0,13.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,28.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Released,306.0,1.0,6.0,1.0,13.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,28.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Released,306.0,1.0,6.0,1.0,13.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()